# Demo - Storing information in EEX 


In [1]:
import eex
import os
import pandas as pd
import numpy as np

In [2]:
# Create empty data layer
dl = eex.datalayer.DataLayer("butane", backend="Memory")
dl.summary()

EEX DataLayer Object

System name: butane
----------------------------------------------
Atom Count:                 0
Bond Count:                 0
Angle Count:                0
Dihedral Count:             0
----------------------------------------------
Number of bond parameters:     0
Number of angle parameters:    0
Number of dihedral parameters: 0
----------------------------------------------


In [3]:
"""
First, we add atoms to the system. Atoms have associated metadata. The possible atom metadata is listed here.

"""

dl.list_valid_atom_properties()


['molecule_index',
 'atom_name',
 'atom_type',
 'atomic_number',
 'charge',
 'xyz',
 'mass',
 'residue_index',
 'residue_name']

In [4]:
"""
TOPOLOGY:

Information can be added to the datalayer in the form of pandas dataframes. Here, we add atom metadata. 

The name of the column corresponds to the atom property.

Populate empty dataframe with relevant information and add to EEX datalayer

"""
# Create empty dataframe
atom_df = pd.DataFrame()

# Create atomic system using pandas dataframe
atom_df["atom_index"] = np.arange(0,4)
atom_df["molecule_index"] = [int(x) for x in np.zeros(4)]
atom_df["residue_index"] = [int(x) for x in np.zeros(4)]
atom_df["atom_name"] = ["C1", "C2", "C3", "C4"]
atom_df["charge"] = np.zeros(4)
atom_df["atom_type"] = [1, 2, 2, 1]
atom_df["X"] = [0, 0, 0, -1.474]
atom_df["Y"] = [-0.4597, 0, 1.598, 1.573]
atom_df["Z"] = [-1.5302, 0, 0, -0.6167]
atom_df["mass"] = [15.0452, 14.02658, 14.02658, 15.0452]

# Add atoms to datalayer
dl.add_atoms(atom_df, by_value=True)

# Print datalayer information
dl.summary()

# Print stored atom properties
dl.get_atoms(properties=None, by_value=True)

EEX DataLayer Object

System name: butane
----------------------------------------------
Atom Count:                 4
Bond Count:                 0
Angle Count:                0
Dihedral Count:             0
----------------------------------------------
Number of bond parameters:     0
Number of angle parameters:    0
Number of dihedral parameters: 0
----------------------------------------------


,molecule_index,atom_name,atom_type,charge,X,Y,Z,mass,residue_index
atom_index,,,,,,,,,
0,0,C1,1,0.0,0.000,-0.4597,-1.5302,15.04520,0
1,0,C2,2,0.0,0.000,0.0000,0.0000,14.02658,0
2,0,C3,2,0.0,0.000,1.5980,0.0000,14.02658,0
3,0,C4,1,0.0,-1.474,1.5730,-0.6167,15.04520,0


In [5]:
"""
TOPOLOGY:

The EEX datalayer now contains four nonbonded atoms. To create butane, atoms must be bonded
to one another.

Add bonds to system

"""

# Create empty dataframes for bonds
bond_df = pd.DataFrame()

# Create column names. Here, "term_index" refers to the bond type index.
# i.e. - if all bonds are the same type, they will have the same term index
bond_column_names = ["atom1", "atom2", "term_index"]

# Create corresponding data. The first row specifies that atom0 is bonded
# to atom 1 and has bond_type id 0
bond_data = np.array([[0, 1, 0,],
        [1, 2, 0],
        [2, 3, 0]])

for num, name in enumerate(bond_column_names):
    bond_df[name] = bond_data[:,num]

dl.add_bonds(bond_df)

In [6]:
dl.summary()

EEX DataLayer Object

System name: butane
----------------------------------------------
Atom Count:                 4
Bond Count:                 3
Angle Count:                0
Dihedral Count:             0
----------------------------------------------
Number of bond parameters:     0
Number of angle parameters:    0
Number of dihedral parameters: 0
----------------------------------------------


In [7]:
"""
TOPOLOGY:

Add angles and dihedrals to system.
"""

# Follow similar procedure as for bonds

angle_df = pd.DataFrame()
dihedral_df = pd.DataFrame()

angle_column_names = ["atom1", "atom2", "atom3", "term_index"]
dihedral_column_names = ["atom1", "atom2", "atom3", "atom4", "term_index"]

angle_data = np.array([[0, 1, 2, 0,], 
                       [1, 2, 3, 0],])

dihedral_data = np.array([[0, 1, 2, 3, 0,]])

for num, name in enumerate(angle_column_names):
    angle_df[name] = angle_data[:,num]

dl.add_angles(angle_df)
    
for num, name in enumerate(dihedral_column_names):
    dihedral_df[name] = dihedral_data[:,num]
    
dl.add_dihedrals(dihedral_df)

In [8]:
dl.summary()

EEX DataLayer Object

System name: butane
----------------------------------------------
Atom Count:                 4
Bond Count:                 3
Angle Count:                2
Dihedral Count:             1
----------------------------------------------
Number of bond parameters:     0
Number of angle parameters:    0
Number of dihedral parameters: 0
----------------------------------------------


## Storing force field information

So far, only the topology and coordinates of the system are specified, and we are not able to calculate an energy.

To calculate the energy, we need to define the functional form of bond, angle, dihedral, and nonbonded interactions and the associated constants.

In this demo, we store the parameters for the TraPPE United Atom forcefield with harmonic bonds.

\begin{equation*}
\ U_{total} = \sum_{bonds}{k_{b}(r-r_{0})^2} + \sum_{angles}{k_{\theta} (\theta - \theta_{eq} )^{2}} + \sum_{dihedrals}{c_{1}[1 + cos(\phi)] + c_{2}[1 - cos(2\phi)] + c_{3}[1 + cos(3\phi)]} + \sum_{i=1}^{N-1}{\sum_{j=i+1}^{N}{ 4\epsilon_{ij}[(\frac{\sigma_{ij}}r_{ij})^{12} - (\frac{\sigma_{ij}}r_{ij})^6] }}
\end{equation*}

In [9]:
"""
EEX FORCE FIELD PARAMETERS

A main component of EEX is internally stored metadata which defines details functional forms including form, constants,
unit types, and default units (if the user does not overrride this option). 

This metadata is stored as human readable dictionaries which can easily be added or maninpulated.
"""

# Here, we examine the metadata present in the bond metadata for a harmonic bond
bond_metadata = eex.metadata.two_body_terms.two_body_metadata

for k, v in bond_metadata["forms"]["harmonic"].items():
    print(k, v)

form K*(r-R0) ** 2
parameters ['K', 'R0']
units {'K': '[energy] * [length] ** -2', 'R0': '[length]'}
description This is a harmonic bond
canonical_form harmonic
utype {'K': <Quantity(1.0, 'kilojoule / angstrom ** 2 / mole')>, 'R0': <Quantity(1, 'angstrom')>}


In [10]:
"""
FORCE FIELD PARAMETERS

To add bonds (or other parameters) using this metadata, the user specifies the form using a keyword ("harmonic") that
matches to EEX's metadata. 

Values for the contstants are passed using a dictionary with the 'parameters' defined in the metadata as keys.

Each bond type is given a uid, and default dimensions may be overwritten by the user using a dictionary 
and the 'utype' argument
"""

# Here, in add_term_parameter, the first argument is the term order. '2'
# corresponds to bonded atoms.

dl.add_term_parameter(2, "harmonic", {'K': 300.9, 'R0': 1.540}, uid=0, utype={'K':"kcal * mol **-1 * angstrom ** -2",
                                                                         'R0': "angstrom"})

0

In [11]:
# If  units or parameters are not compatible with the metadata, the datalayer will not allow storage of the parameter.
# Here, we have changed 'K' to simply "kcal". This will fail (uncomment to test)

#dl.add_term_parameter(2, "harmonic", {'K': 300.9, 'R0': 1.540}, uid=0, utype={'K':"kcal",'R0': "angstrom"})

In [12]:
## Add harmonic angle parameters
dl.add_term_parameter(3, "harmonic", {'K': 62.100, 'theta0': 114}, uid=0, utype={'K':'kcal * mol ** -1 * radian ** -2',
                                                                             'theta0': 'degree'})

# Add OPLS dihedral parameter
dl.add_term_parameter(4, "opls", {'K_1': 1.41103414, 'K_2': -0.27101489, 
                                  'K_3': 3.14502869, 'K_4': 0}, uid=0, utype={'K_1': 'kcal * mol ** -1',
                                                                               'K_2': 'kcal * mol ** -1',
                                                                               'K_3': 'kcal * mol ** -1',
                                                                               'K_4': 'kcal * mol ** -1'})

0

In [13]:
"""
NONBOND PARAMETERS

For nonbond parametets, we currently provide support for Lennard Jones and Buckingham potentials

Most programs use pair-wise Lennard Jones potentials for nonbond interactions. Our internal metadata stores these as A
and B parameters. However, uses may specify other forms such as epsilon/sigma, epsilon, Rmin, etc.

Lennard Jones parameters can be added as a pair (atom_type1, atom_type2) or for a single atom type with a mixing rule.

"""

dl.add_nb_parameter(atom_type=1, nb_name="LJ", 
                    nb_model="epsilon/sigma", nb_parameters={'sigma': 3.75, 'epsilon': 0.1947460018}, 
                    utype={'sigma': 'angstrom', 'epsilon': 'kcal * mol ** -1'})

dl.add_nb_parameter(atom_type=2, nb_name="LJ", 
                    nb_model="epsilon/sigma", nb_parameters={'sigma': 3.95, 'epsilon': 0.0914112887},
                    utype={'sigma': 'angstrom', 'epsilon': 'kcal * mol ** -1'})

dl.set_mixing_rule('arithmetic')

In [14]:
# Retrieve stored parameters

print("All stored parameters\n", dl.list_nb_parameters("LJ"), "\n\n")

# To apply the mixing rule:
dl.build_LJ_mixing_table()

print("All stored parameters\n", dl.list_nb_parameters("LJ"), "\n\n")

# These can also be retrieved for only single atoms, or for atom pairs by using itype='single' or itype='pairs'
pair_interactions = dl.list_nb_parameters("LJ", itype="pair")

print("Pair parameters\n", pair_interactions)

All stored parameters
 {(1, None): {'A': 25205506.57062631, 'B': 9063.748030795972}, (2, None): {'A': 22070727.37761266, 'B': 5810.775176118389}} 


All stored parameters
 {(1, None): {'A': 25205506.57062631, 'B': 9063.748030795972}, (2, None): {'A': 22070727.37761266, 'B': 5810.775176118389}, (1, 1): {'A': 25205506.570626277, 'B': 9063.748030795967}, (1, 2): {'A': 23681794.71896144, 'B': 7271.9391332462965}, (2, 2): {'A': 22070727.377612628, 'B': 5810.775176118385}} 


Pair parameters
 {(1, 1): {'A': 25205506.570626277, 'B': 9063.748030795967}, (1, 2): {'A': 23681794.71896144, 'B': 7271.9391332462965}, (2, 2): {'A': 22070727.377612628, 'B': 5810.775176118385}}


Alternatively, these could have been set directly as pairs without a mixing rule.

`# Add NB parameters with pairs`

`dl.add_nb_parameter(atom_type=1, atom_type2=1, nb_name="LJ", nb_model="AB", nb_parameters=[1.0, 1.0])`

`dl.add_nb_parameter(atom_type=1, atom_type2=2, nb_name="LJ", nb_model="epsilon/sigma", nb_parameters=[1.0, 1.0])`

`dl.add_nb_parameter(atom_type=2, atom_type2=2, nb_name="LJ", nb_model="epsilon/sigma", nb_parameters=[1.0, 1.0])`




In [15]:
dl.summary()

EEX DataLayer Object

System name: butane
----------------------------------------------
Atom Count:                 4
Bond Count:                 3
Angle Count:                2
Dihedral Count:             1
----------------------------------------------
Number of bond parameters:     1
Number of angle parameters:    1
Number of dihedral parameters: 1
----------------------------------------------


In [16]:
# Evaluate system energy
energy_system1 = dl.evaluate(utype="kcal * mol ** -1")

print(energy_system1)

{'two-body': 3.028486967052281, 'three-body': 12.727397850233963, 'four-body': 0.8605450603579679, 'total': 16.616429877644215}


## Reading from MD input files

Typically, this storage would not be done by hand as shown above. Instead, readers and writers for specific softwares are used.

Below, we use the plugin for AMBER with EEX to read in an amber file with information for the a butane molecule which is equivalent to the one created in the first datalayer. The EEX translator uses the functions displayed above to store all information from the amber prmtop and inpcrd files in the datalayer.

In [18]:
# Preview an amber prmtop (parameter-topology file) for Amber.
butane_file = os.path.join("..", "eex", "data", "amber","alkanes", "trappe_butane_single_molecule.prmtop")

f = open(butane_file)
print(f.read())
f.close()

%VERSION  VERSION_STAMP = V0001.000  DATE = 01/31/18  12:29:16                  
%FLAG TITLE                                                                     
%FORMAT(20a4)                                                                   
Cpptraj                                                                         
%FLAG POINTERS                                                                  
%FORMAT(10I8)                                                                   
       4       2       0       3       0       2       0       4       0       0
       7       1       3       2       4       2       1       4       2       0
       0       0       0       0       0       0       0       1       4       0
       0
%FLAG ATOM_NAME                                                                 
%FORMAT(20a4)                                                                   
C1  C2  C3  C4  
%FLAG CHARGE                                                                    
%F

In [19]:
# Create new datalayer and populate using amber reader

dl_amber = eex.datalayer.DataLayer("butane_amber")
eex.translators.amber.read_amber_file(dl_amber, butane_file)

dl_amber.summary()

EEX DataLayer Object

System name: butane_amber
----------------------------------------------
Atom Count:                 4
Bond Count:                 3
Angle Count:                2
Dihedral Count:             4
----------------------------------------------
Number of bond parameters:     2
Number of angle parameters:    1
Number of dihedral parameters: 4
----------------------------------------------


## Comparing the two datalayers
The summary shows a system with 4 atom, 3 bonds, 2 angles and 4 dihedrals.  This differs from the first datalayer in the number of dihedrals and the number of dihedral parameters. However, the evaluated system energy is equivalent.

This is because AMBER stores dihedral angles using a different functional form. Instead of a single equation, dihedrals with multiple terms are built from multiple harmonic equations. The equations are equivalent when evaluated.

\begin{equation*} 
\sum_{dihedrals}{(V_{0}[1 + cos(0\phi)]}) + \sum_{dihedrals}{(V_{1}[1 + cos(n\phi)]}) + \sum_{dihedrals}{(V_{2}[1 + cos(2\phi - \pi)]}) + \sum_{dihedrals}{(V_{3}[1 + cos(3\phi )]}) = \sum_{dihedrals}{c_{0} + c_{1}[1 + cos(\phi)] + c_{2}[1 - cos(2\phi)] + c_{3}[1 + cos(3\phi)]}
\end{equation*}

Although not not implemented yet, EEX should eventually be able to identify and perform a translation between equivalent functional forms.

In [20]:
energy_system2 = dl_amber.evaluate(utype="kcal * mol ** -1")

In [21]:
for k in energy_system1:
    energy_difference = energy_system1[k] - energy_system2[k]
    print(k," difference:\t %.3f" % energy_difference)

two-body  difference:	 -0.000
three-body  difference:	 -0.000
four-body  difference:	 0.000
total  difference:	 -0.000


In [22]:
# Compare stored NB parameters
eex.testing.dict_compare(dl_amber.list_nb_parameters("LJ"), dl.list_nb_parameters("LJ", itype="pair"))

True

# Writing output files

In [23]:
# We can now write the amber file we read for lammps.
eex.translators.lammps.write_lammps_file(dl_amber, "output_lammps.data", "output_lammps.in")

# Write a local copy of the amber datalayer for amber.
eex.translators.amber.write_amber_file(dl_amber, "amber_output.prmtop")

0

In [24]:
## Read the written file into a datalayer ##

dl_lammps = eex.datalayer.DataLayer("butane_lammps")
eex.translators.lammps.read_lammps_input_file(dl_lammps, "output_lammps.in")

In [25]:
f = open("output_lammps.data")
print(f.read())
f.close()

LAMMPS data file generated by MolSSI EEX

 4 atoms
 2 atom types
 3 bonds
 2 bond types
 2 angles
 1 angle types
 4 dihedrals
 4 dihedral types
 0 impropers
 0 improper types
 0.000000 100.000000 xlo xhi
 0.000000 100.000000 ylo yhi
 0.000000 100.000000 zlo zhi


PairIJ Coeffs

 1  1 0.19474595 3.75000000
 1  2 0.13342411 3.85000000
 2  2 0.09141137 3.95000000

Bond Coeffs

 1 300.90000000 1.54000000
 2 300.90000000 1.54000000

Angle Coeffs

 1 62.10011250 114.00004886

Dihedral Coeffs

 1 0.00000000 0.00000000 0.00000000
 2 0.70551689 1.00000000 0.00000000
 3 -0.13550741 2.00000000 -180.00007714
 4 1.57251395 3.00000000 0.00000000

 Masses

1 15.0452
2 14.02658

 Atoms

1  1  1 0.00000000  0.00000000 -0.45970000 -1.53020000
2  1  2 0.00000000  0.00000000  0.00000000  0.00000000
3  1  2 0.00000000  0.00000000  1.59800000  0.00000000
4  1  1 0.00000000 -1.47400000  1.57300000 -0.61670000

 Bonds

1 1 1 2
2 2 2 3
3 1 3 4

 Angles

1 1 1 2 3
2 1 2 3 4

 Dihedrals

1 1 1 2 3 4
2 2 1 2 3 4


In [26]:
lammps_energy = dl_lammps.evaluate(utype="kcal * mol ** -1")

In [27]:
# Compare energies
for k in energy_system1:
    energy_difference = lammps_energy[k] - energy_system2[k]
    print(k," difference:\t %.3f" % energy_difference)

two-body  difference:	 0.000
three-body  difference:	 0.000
four-body  difference:	 -0.000
total  difference:	 0.000


# Translating small peptide structure

Here, we demonstrate translating a small solvated peptide structure (from http://ambermd.org/tutorials/basic/tutorial0/index.htm) to LAMMPS using EEX.

In [29]:
dl_dna = eex.datalayer.DataLayer("DNA_amber")

DNA_file = os.path.join("..", "eex", "data", "amber","peptides", "alanine_dipeptide.prmtop")

eex.translators.amber.read_amber_file(dl_dna, DNA_file)

dl_dna.summary()

EEX DataLayer Object

System name: DNA_amber
----------------------------------------------
Atom Count:                 1912
Bond Count:                 1911
Angle Count:                36
Dihedral Count:             67
----------------------------------------------
Number of bond parameters:     13
Number of angle parameters:    21
Number of dihedral parameters: 20
----------------------------------------------


In [30]:
eex.translators.lammps.write_lammps_file(dl_dna,"lammps_ala.data", "lammps_ala.in")

True

In [31]:
f = open("lammps_ala.data")
print(f.read())
f.close()

LAMMPS data file generated by MolSSI EEX

 1912 atoms
 9 atom types
 1911 bonds
 13 bond types
 36 angles
 21 angle types
 67 dihedrals
 20 dihedral types
 0 impropers
 0 improper types
 0.000000 31.397856 xlo xhi
 0.000000 34.100045 ylo yhi
 0.000000 29.272966 zlo zhi


PairIJ Coeffs

 1  1 0.01570000 2.64953279
 1  2 0.04144370 3.02460115
 2  2 0.10940000 3.39966951
 1  3 0.03674507 3.02460115
 2  3 0.09699691 3.39966951
 3  3 0.08600000 3.39966951
 1  4 0.05741951 2.80472734
 2  4 0.15157177 3.17979570
 3  4 0.13438750 3.17979571
 4  4 0.21000000 2.95992190
 1  5 0.05166237 2.94976566
 2  5 0.13637448 3.32483402
 3  5 0.12091319 3.32483402
 4  5 0.18894444 3.10496021
 5  5 0.17000000 3.24999852
 1  6 0.01570000 1.85930563
 2  6 0.04144370 2.23437398
 3  6 0.03674507 2.23437399
 4  6 0.05741951 2.01450018
 5  6 0.05166237 2.15953849
 6  6 0.01570000 1.06907846
 1  7 0.01570000 2.56044292
 2  7 0.04144370 2.93551128
 3  7 0.03674507 2.93551128
 4  7 0.05741951 2.71563747
 5  7 0.05166